# Run ROHF

In [2]:
import pyscf.tools

from orbitalpartitioning import *


molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
 """

basis = "def2-tzvp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()

print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "chkfile"
dm = np.load("rohf_dm.npy")
mf.run(dm)
##################################################################

#
#   Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]
ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]



symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = chkfile
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5357.2923512482
HOMO (B) = 0.00686403549231895  LUMO (A) = 0.356423407071967
cycle= 1 E= -5357.29235124816  delta_E= 4e-11  |g|= 5.12e-06  |ddm|= 9.48e-06
HOMO (B) = 0.00686371887952404  LUMO (A) = 0.356423322963218
Extra cycle  E= -5357.29235124753  delta_E= 6.31e-10  |g|= 2.23e-05  |ddm|= 4.01e-05
converged SCF energy = -5357.29235124753


In [3]:
K = mf.get_k()

In [62]:

# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
d0 = []
 
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        # print(ao)
        if ao[2] in ("3d","4d","5d","6d","7d"):
        # if ao[2] in ("3d","4d","5d"):
            d0.append(ao_idx)
        if ao[2] in ("3d","4d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("2p","3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3d","4d"):
            frag3.append(ao_idx)
            full.append(ao_idx)

X = scipy.linalg.sqrtm(S)
Xinv = np.linalg.inv(X)

Pd = np.eye(S.shape[0])[:,d0]
Fd = (Xinv.T@F@Xinv)
de,dU = np.linalg.eigh(Pd.T@Fd@Pd)
for i in de:
    print(" d-orbital energy: %12.8f"%i)

dU = Xinv@Pd@dU
print(dU.shape)
print(np.linalg.det(dU.T @ S @ dU))
print(dU.shape)
pyscf.tools.molden.from_mo(mf.mol, "Cdu.molden", dU)
# pyscf.tools.molden.from_mo(mf.mol, "Cdu.molden", S@Xinv@Pd)
print(Pd.T@dU[:,6])
frags = [frag1, frag2, frag3]
print(frags)

 d-orbital energy:  -0.07405967
 d-orbital energy:  -0.06999190
 d-orbital energy:  -0.05756073
 d-orbital energy:  -0.05385004
 d-orbital energy:  -0.04027186
 d-orbital energy:   0.52364099
 d-orbital energy:   0.54632206
 d-orbital energy:   0.59082944
 d-orbital energy:   0.67015684
 d-orbital energy:   0.83353794
 d-orbital energy:   1.40109859
 d-orbital energy:   1.44419768
 d-orbital energy:   1.46725310
 d-orbital energy:   1.51531596
 d-orbital energy:   1.54626271
 d-orbital energy:   4.12420701
 d-orbital energy:   4.17045368
 d-orbital energy:   4.18426892
 d-orbital energy:   4.22289622
 d-orbital energy:   4.29643743
(343, 20)
1.0000000000001124
(343, 20)

WARN: orbitals [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] not symmetrized, norm = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5]

[-7.51801906e-02  5.76799338e-02  7.15840250e-03 -2.95956468e-02
  9.45107914e-02 -3.50189024e-02  3.38781542e-02  1.76561002e-03
 -9.2